In [ ]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import glob
import wandb

sns.set()

# matplotlib.use("pgf")
matplotlib.rcParams.update(
    {
        "pgf.texsystem": "pdflatex",
        "font.family": "serif",
        "font.serif": "Palatino",
        "text.usetex": True,
        "pgf.rcfonts": False,
    }
)

plot_dir = "/Users/vaclav/prog/thesis/figures/"

In [ ]:
models = [
    # default params
    # {"dataset": "Violin", "name": "0725-ddspae-cnn-1", "display_name": "Ours-default"},
    # {"dataset": "Trumpet", "name": "0804-ddspae-cnn-3", "display_name": "Ours-default"},
    # ch=16
    # {"dataset": "Violin", "name": "0809-ddspae-cnn-5", "display_name": "Ours-tiny"},
    # {"dataset": "Trumpet", "name": "0809-ddspae-cnn-4", "display_name": "Ours-tiny"},
    #     # inverted bottleneck, ch=32
    # {"dataset": "Violin", "name": "0726-ddspae-cnn", "display_name": "Ours-IB"},
    # {"dataset": "Trumpet", "name": "0809-ddspae-cnn", "display_name": "Ours-IB"},
]

models2 = []
for m in models:
    m = m.copy()
    m["quantization"] = False
    models2.append(m)
    m = m.copy()
    m["quantization"] = True
    models2.append(m)

models = models2

eval_dir = "/Users/vaclav/prog/thesis/data/eval_data/"

api = wandb.Api()

for model in models:
    model_name = model["name"]

    suffix = "rtq" if model["quantization"] else "rt"
    model_eval_name = f"{model_name}-{suffix}"

    artifact = api.artifact(
        # TODO: revert from v0 to latest!!!!!!!!!!!!!!!!!!!
        f"neural-audio-synthesis-thesis/nas-evaluation/eval-{model_eval_name}:v0"
    )
    csv_dir = artifact.checkout()

    run = artifact.logged_by()

    model["csv_path"] = os.path.join(csv_dir, f"eval-{model_eval_name}.csv")

    model["loss"] = run.summary["losses/total_loss"]
    
    df = pd.read_csv(model["csv_path"], index_col=0)
    model["decoder_inference_time"] = df["Autoencoder.decoder"].mean()

    print(artifact.created_at, model_eval_name)

In [ ]:
df = pd.DataFrame(
    [
        [
            m["display_name"],
            m["quantization"],
            m["dataset"],
            m["loss"],
            m["decoder_inference_time"] * 1000,
        ]
        for m in models
    ],
    columns=["Model", "Quantization", "Dataset", "Loss", "Decoder inference time (ms)"],
)
df = df.pivot(index=["Model", "Quantization"], columns="Dataset")
# df.columns = df.columns.get_level_values(1)
# df.columns.name = None
# df = df.reset_index()

In [ ]:
df["Loss"]

In [ ]:
df["Decoder inference time (ms)"]

In [ ]:
run.summary["prediction_time_secs"]

In [ ]:
for a in run.logged_artifacts():
    if "time_distribution_table" in a.name:
        path = a.checkout()
        break

In [ ]:
csv_dir